# LLM model

In [ ]:
!pip install gradio langchain openai langchain-community

In [4]:
import os
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.schema import BaseMessage
from openai import OpenAI
import tiktoken
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory


# === CONFIG ===
os.environ["OPENAI_API_KEY"] = "microsoft/phi-4"
os.environ["OPENAI_API_BASE"] = "http://127.0.0.1:1234/v1"

# === MODEL ===
llm = ChatOpenAI(model="microsoft/phi-4", temperature=0.7)


# === PROMPT TEMPLATE ===
memory = ConversationBufferWindowMemory(k=5, return_messages=True)

template = ChatPromptTemplate.from_messages([
    ("system", "คุณคือผู้ช่วย AI ที่ใจดีและเป็นประโยชน์"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=template,
    verbose=False,
)

# === TOKEN COUNTER ===
def count_tokens(text, model="microsoft/phi-4"):
    try:
        encoding = tiktoken.encoding_for_model(model)
    except:
        encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(text))

# === CHATBOT HANDLER ===
def chatbot_interface(user_input, history):
    # Predict
    response = conversation.predict(input=user_input)

    # Count tokens
    input_tokens = count_tokens(user_input)
    output_tokens = count_tokens(response)
    total_tokens = input_tokens + output_tokens

    # Memory count
    memory_count = len(memory.chat_memory.messages) // 2  # user+bot = 1 exchange

    # Add to history
    history.append((user_input, f"{response}\n\n🧠 Tokens: {total_tokens}, Memory turns: {memory_count}"))

    return "", history

# === BUILD INTERFACE ===
def build_interface():
    with gr.Blocks() as demo:
        gr.Markdown("## 🤖 LangChain + LM Studio Chatbot")
        chatbot = gr.Chatbot()
        msg = gr.Textbox(label="พิมพ์ข้อความที่นี่...")
        submit_btn = gr.Button("📨 ส่งข้อความ")
        clear = gr.Button("🔁 ล้างบทสนทนา")

        state = gr.State([])

        submit_btn.click(chatbot_interface, [msg, state], [msg, chatbot])
        msg.submit(chatbot_interface, [msg, state], [msg, chatbot])
        clear.click(lambda: [], None, chatbot)

    return demo

# === RUN ===
if __name__ == "__main__":
    demo = build_interface()
    demo.launch(inbrowser=True)


C:\Users\FOth-PC\AppData\Local\Temp\ipykernel_21640\739071006.py:66: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
